In [1]:
"""
developing code for taking
- dataset, with parses (pre extracted)
- building "planner" models (new BehModel classes)
- combining into a behmodelhandler, which is general purpose

UPDATED: finalized stuff - actually running modeling, traiing models etc, is moved to:
devo_taskmodel_finalized_071721.ipynb

[OBSOLETE] - This was used to first develop Task modeling code. It combines (i) Motor Cost and (ii) Dataset task model stuff.
Nothing here is up to date, some are sort of idfferent branches compared to what I ended up doing. 
But hihgly unlikely to use anything here.

"""


'\ndeveloping code for taking\n- dataset, with parses (pre extracted)\n- building "planner" models (new BehModel classes)\n- combining into a behmodelhandler, which is general purpose\n\nUPDATED: finalized stuff - actually running modeling, traiing models etc, is moved to:\ndevo_taskmodel_finalized_071721.ipynb\n\n[OBSOLETE] - This was used to first develop Task modeling code. It combines (i) Motor Cost and (ii) Dataset task model stuff.\nNothing here is up to date, some are sort of idfferent branches compared to what I ended up doing. \nBut hihgly unlikely to use anything here.\n\n'

In [ ]:
# # ## OUTLINE


# Tasks module (holds information for this task)

# Behavior module (holds behavior on this task)

# Parses module
# - holds all parses for a given task.
# - Representations:
#     - strokes
#     - symbolic features (number strokes)
#     - graph control pts.
#     - [optional: consider direction?]

# Model module
# - Flexible holding of params, can train given new set of tasks, if needed.
#     - Prior scorer
#     - Likeli scorer
#     - Posterior scorer
# - can score any input parses, P(parse | prior)
#     - Parses represented both as strokes and graph control pts (and objects ?)
#     - INPUT: All kinds of models
#         - ActionGrammar (log linear version)
#             - MotorCost
#             - [Flatten out MotorCost so that Action Grammar is flat]
#             - [Each of these combined in log linear model]
#         - Stroke manifold model (or other form of learned action primitive)
#         - GNS
# - can score any behavior, P(beh|parse)
#     - INPUT: scoring functions (likeli)
#         - 
# - can get summary score (posterior)
#     - INPUT: flexible posterior function
        
        



# # Structure of current taskmodel

# Model
#     - parser
#     - prior fun
#     - prior norm
#     - likelis
#     - posterior

# Dataset
# - data
# - Model
# - plots/summaries

# getprior
# getlikeli




# OLD CODE

#### Code from motor cost (devo_taskmodel_motorcost_010321)

In [ ]:
# 0) New MC model
from pythonlib.drawmodel.efficiencycost import Cost
MC = Cost()

# 1) extract probedat anew
Probedat = ProbedatTaskmodel(PROBEDAT, MD)
if False:
    filtdict = {
    #     "date":["200930", "201001"]}
        "date":["200928", "200929", "200930", "201001"]}
    Probedat.filterProbedat(filtdict);

# == define prior function based on MC model
def priorFunction(p, trial):
    return MC.score(p["strokes"], trial["task"])

# 2) apply mdoel
params_data, params_model = Probedat.getParams()
if False:
    params_model["priorver"] = priorFunction
params_data["standardize"]["standardize_strokes"] = True

Probedat.applyModel(params_data, params_model)
Probedat.Datamodel.plotPosteriorHist()

#### Code from dataset version 

In [ ]:
# 2) model-based features (task model score)
from pythonlib.dataset.analy import taskmodel_assign_score
taskmodel_assign_score(Dthis)

# model summary score
def F(x):
    """compare two models, retgurns index between -1,1
    """   
    a = x["MOD_3line"]
    b = x["MOD_linePlusL"]
    return 2*((a/(a+b))-0.5)
from pythonlib.tools.pandastools import applyFunctionToAllRows
Dthis.Dat = applyFunctionToAllRows(Dthis.Dat, F, "modelcomp_offline")
# FEATURE_NAMES = list(set(FEATURE_NAMES+ ["MOD_3line", "MOD_linePlusL", "mod2_minus_mod1", "modelcomp_offline"]))
FEATURE_NAMES = list(set(FEATURE_NAMES+ ["MOD_3line", "MOD_linePlusL", "modelcomp_offline"]))

# NEW CODE

In [ ]:
# Load dataset
from pythonlib.dataset.dataset import Dataset
import matplotlib.pyplot as plt

### Load a dataset, to have tasks to test
animal = "Red"
expt = "lines5"
rule = "straight"
D = Dataset([])
D.load_dataset_helper(animal, expt, rule=rule)
D.load_tasks_helper(convert_coords_to_abstract=False)


In [ ]:
# keep only fixed tasks
D.filterPandas({"random_task":[False]}, "modify")
# Limit to subset of trials, just for devo reasons
D.subsampleTrials(1,1)

In [ ]:
# Load presaved parses
# keep only fixed tasks
D.filterPandas({"random_task":[False]}, "modify")

list_parse_params = [
    {"quick":True, "ver":"graphmod", "savenote":"fixed_True"},
    {"quick":True, "ver":"nographmod", "savenote":"fixed_True"}]

list_suffixes = ["graphmod", "nographmod"]

    
D.parser_load_presaved_parses(list_parse_params, list_suffixes)

In [ ]:
# Finalize strokes


#### TODO: Quick way to extract all unique parses from multiple Parsers
iterate over all pairs across Parsers. no need to iterate within since already checked that they are unique

In [ ]:
# Extract parses
import random
ind = random.randint(0, len(D.Dat))
list_colnames = [name for name in D.Dat.columns if "parser_" in name]

list_parsers = [D.Dat.iloc[ind][name] for name in list_colnames]

P1 = list_parsers[0]
P2 = list_parsers[1]

# MERGE PARSERS
# if graphs are the same, 
print(P1.Graph == P2.Graph)


P1.plot_graph()
P2.plot_graph()

P1.print_graph()
P2.print_graph()

P1.summarize_parses()
P2.summarize_parses()

In [ ]:
# Likeli functions
from pythonlib.drawmodel.strokedists import distscalarStrokes



#### Continue analysis

In [ ]:
# Construct model
from pythonlib.behmodel.scorer.scorer import Scorer
import numpy as np
from pythonlib.drawmodel.efficiencycost import Cost


######### PRIOR
# == define prior function based on MC model
MC = Cost()
Pr = Scorer()
# def F(strokes):
#     return MC.score(strokes)
# def F(strokes):
#     return np.random.rand()
def F(strokes):
    return score_function([strokes], ver="ink", normalization = "negative", test=False,
                  use_torch=False, origin=None)
Pr.input_score_function(F)

######## LIKELI
from pythonlib.drawmodel.strokedists import distscalarStrokes
Li = Scorer()
def F(beh, parse):
    return distscalarStrokes(beh, parse, "dtw_segments")
#     return np.random.rand()
Li.input_score_function(F)

######## POSTERIOR
Po = Scorer()
def F(likelis, priors):
    return np.dot(likelis, priors)
#     return np.random.rand()
Po.input_score_function(F)


######## TASK (task and parses)
from pythonlib.behmodel.score_dataset import prepare_trial
ind=1
Beh, Task = prepare_trial(D, ind)

###### MODEL
from pythonlib.behmodel.behmodel import BehModel

BM = BehModel()
BM.input_model_components(Pr, Li, Po)
BM.score_single_trial(Beh, Task)

# Model module
# - Flexible holding of params, can train given new set of tasks, if needed.
#     - Prior scorer
#     - Likeli scorer
#     - Posterior scorer
# - can score any input parses, P(parse | prior)
#     - Parses represented both as strokes and graph control pts (and objects ?)
#     - INPUT: All kinds of models
#         - ActionGrammar (log linear version)
#             - MotorCost
#             - [Flatten out MotorCost so that Action Grammar is flat]
#             - [Each of these combined in log linear model]
#         - Stroke manifold model (or other form of learned action primitive)
#         - GNS
# - can score any behavior, P(beh|parse)
#     - INPUT: scoring functions (likeli)
#         - 
# - can get summary score (posterior)
#     - INPUT: flexible posterior function
        

In [ ]:
P.extract_all_parses_as_list("summary")[0][2]

In [ ]:
A = D.parser_list_of_parses(1)

In [ ]:
### visualize each parse

def plotStrokes(strokes, ax):
    """ wrapper"""
    from pythonlib.drawmodel.strokePlots import plotDatStrokes

#     plotDatStrokes(strokes, ax=ax, plotver="order", add_stroke_number=False, each_stroke_separate=False)
    plotDatStrokes(strokes, ax=ax, clean_ordered=True)
    
def plotExampleTrial(BehMod, max_parses = 34, sort_by = None):
    """ useful plots of beahviora nd scores/priors/likelies, for a given trial
    trial inds are from 0, 1, 2, ... (i.e,. not to original trial indices in 
    filedata)"""
    print("NOTE: the x and y lims are hacky, should change")

    ncols = 6
    Parser = BehMod._task.Parser
    nparse = len(Parser.Parses)
    nparse = min(max_parses, nparse)
    nrows = int(np.ceil((nparse+2)/ncols))

    # plt.figure(figsize=(ncols*3,nrows*3))
    fig, axes = plt.subplots(nrows = nrows, ncols=ncols, 
        figsize=(ncols*3,nrows*3), sharex=True, sharey=True, squeeze=False)
    
    # 1) ==== BEHAVIOR
    strokes_beh = BehMod._behavior.Strokes
#     strokes_beh = self.trials[trialind]["behavior"]["strokes"]
    # ax = plt.subplot(nrows, ncols, 1)
    ax = axes[0,0]
    # ax = plotTrialSimple(filedata, 1, ax=ax, plotver="empty", nakedplot=True, plot_task_stimulus=False, plot_drawing_behavior=False)
    ax.plot(1,1)
    plotStrokes(strokes_beh, ax)
    # plt.xlim([-400, 400])
    # plt.ylim([-600, 600])
    ax.set_title("behavior")

    # 2) ==== TASK
#     strokes = self.trials[trialind]["task"]["strokes"]
    post = BehMod._posterior_score
#     post = self.trials[trialind]["posterior"]
    # ax = plt.subplot(nrows, ncols, 2)
#     ax = axes[0,1]
#     plotStrokes(strokes, ax)
#     ax.plot(1,1,'o')
    # plt.xlim([-400, 400])
    # plt.ylim([-600, 600])
#     print(post)
# #     ax.set_title(f"task|post={post:.2f}")
#     ax.set_title(f"task|post={post}")
    
    parses = [Parser.Parses[i]["strokes"] for i in range(len(Parser.Parses))]
    likelis = BehMod._likeli_scores
    priors = BehMod._prior_scores
#     posteriors = BehMod._post_scores
    tmp = [(par, li, pr) for par, li, pr in zip(parses, likelis, priors)]
#     parses = self.trials[trialind]["model_parses"]
    if sort_by is not None:
        if sort_by == "likeli":
            # print(sorted(parses, key=lambda x:x["likeli"])[0])
            tmp = sorted(tmp, key=lambda x:-x[1])
        elif sort_by == "prior":
            tmp = sorted(tmp, key=lambda x:-x[2])
        else:
            print(sort_by)
            assert False, "not coded"
        parses = [t[0] for t in tmp]
        likelis = [t[1] for t in tmp]
        priors = [t[2] for t in tmp]

    for i, (p, li, pr) in enumerate(zip(parses, likelis, priors)):
        if i<max_parses:
            ax = axes.flatten()[i+2]
            # ax = plt.subplot(nrows,ncols,i+3)
            plotStrokes(p, ax)
            
#             ax.set_title(f"pr{pr:.2f}, li{li:.2f}")
            ax.set_title(f"pr{pr}, li{li}")
            
            # plt.xlim([-400, 400])
            # plt.ylim([-400, 400])
            
plotExampleTrial(BM, sort_by="likeli")


In [ ]:
# score not just one trial, but entire dataset, using a single model.

score_list = []
for i in range(len(D.Dat)):
    if i%10==0: 
        print(i)
    Beh, Task = prepare_trial(D, i)
    score_list.append(BM.score_single_trial(Beh, Task))
#     assert False


In [ ]:
# Plot top parses based on prior, likeli.


In [ ]:
# ### TODO:
# 1) motor cost model is very slow...
# 2) scorer can normalize scores to probabilities (softmax)


#### Feature Extractor
Single parse (ind in Parser object) --> list of scalar features

NOTE: This builds on efficiencycost model (pulls in those things here)

In [ ]:
# for debugging.
D.Dat.iloc[indtrial]["parser_nographmod"].Parses = D.Dat.iloc[indtrial]["parser_nographmod"].Parses[:1]
D.Dat.iloc[indtrial]["parser_graphmod"].Parses = D.Dat.iloc[indtrial]["parser_graphmod"].Parses[:1]

In [ ]:
# Takes in a Dataset row, and outputs list of scores, one for each parse..

# Prior
from pythonlib.behmodel.scorer.prior_functions import prior_feature_extractor
Pr = prior_feature_extractor()

# Likeli
from pythonlib.behmodel.scorer.likeli_functions import likeli_dataset
Li = likeli_dataset()

# Post
Po = Scorer()
def F(likelis, priorprobs):
    from pythonlib.behmodel.scorer.utils import posterior_score
    postscore = posterior_score(likelis, priorprobs, "weighted")
    return postscore
Po.input_score_function(F)

###### MODEL
from pythonlib.behmodel.behmodel import BehModel
indtrial = 6
BM = BehModel()
BM.input_model_components(Pr, Li, Po)
BM.score_single_trial_dataset(D, indtrial)

In [ ]:
BM.plot_scatter_likeli_prior()

N = 5
sort_by = "prior"
BM.plot_sorted_by(sort_by, N)
sort_by = "likeli"
BM.plot_sorted_by(sort_by, N)


### GET SCORES ACROSS ALL TRIALS


In [ ]:
from pythonlib.behmodel.score_dataset import score_dataset
from pythonlib.behmodel.scorer.prior_functions import prior_feature_extractor
from pythonlib.behmodel.scorer.likeli_functions import likeli_dataset
from pythonlib.behmodel.scorer.poster_functions import poster_dataset
from pythonlib.behmodel.behmodel import BehModel
from pythonlib.behmodel.scorer.scorer import Scorer


In [ ]:
# Load dataset
from pythonlib.dataset.dataset import Dataset
import matplotlib.pyplot as plt

SDIR = "/data2/analyses/main/model_comp/planner"

### Load a dataset, to have tasks to test
animal = "Red"
expt = "lines5"
for rule in ["straight", "bent"]:
    D = Dataset([])
    D.load_dataset_helper(animal, expt, rule=rule)
    D.load_tasks_helper()
    D.filterPandas({"random_task":[False]}, "modify")
    list_parse_params = [
        {"quick":True, "ver":"graphmod", "savenote":"fixed_True"},
        {"quick":True, "ver":"nographmod", "savenote":"fixed_True"}]
    list_suffixes = ["graphmod", "nographmod"]
    D.parser_load_presaved_parses(list_parse_params, list_suffixes)
    
    for rule_model in ["straight", "bent"]:
        
        # Prior
        Pr = prior_feature_extractor(rule=rule_model)
        # Likeli
        Li = likeli_dataset()
        # Post
        Po = Scorer()
        def F(likelis, priorprobs):
            from pythonlib.behmodel.scorer.utils import posterior_score
            postscore = posterior_score(likelis, priorprobs, "weighted")
            return postscore
        Po.input_score_function(F)
        # Model
        BM = BehModel()
        BM.input_model_components(Pr, Li, Po)

        # savedir
        sdir = f"{SDIR}/pilot/dset_{D.identifier_string()}-vs-mod_{rule_model}"
        import os
        os.makedirs(sdir, exist_ok=True)
        
        # RUN
        score_dataset(D, BM, saveon=True, sdir=sdir)


In [ ]:

# Holder, given multiple datasets and multiple BMs.


# Subsample parses, e.,g to top-likel, bottom-likeli, and uniform in between


### Load pre-computed scores (for each dataset - model pair) and combine


In [ ]:

# Load datset, extract its score for each model.
animal = "Red"
expt = "lines5"
Dlist = []
for rule in ["straight", "bent"]:
    D = Dataset([])
    D.load_dataset_helper(animal, expt, rule=rule)
    D.load_tasks_helper()
    D.filterPandas({"random_task":[False]}, "modify")    
    Dlist.append(D)
    
    for rule_model in ["bent", "straight"]:
        model_id = ["pilot", rule_model]
        
        D.parser_load_precomputed_posteriors(model_id)
        
    
# Combine multiple datasets (must have same models)

# Plot scores

In [ ]:
from pythonlib.dataset.dataset import concatDatasets
D = concatDatasets(Dlist)

In [ ]:
D.Dat["parser_postscores_pilot_bent"].hist()
D.Dat["parser_postscores_pilot_straight"].hist()

In [ ]:
D.Dat.iloc[0]

In [ ]:
# make sure no repeated trialcodes
assert len(D.Dat["trialcode"].unique().tolist()) == len(D.Dat)


In [ ]:
# preprocess to get groups
from pythonlib.dataset.analy import preprocessDat

D, GROUPING, GROUPING_LEVELS, FEATURE_NAMES, SCORE_COL_NAMES = preprocessDat(D, "lines5")

In [ ]:
SCORE_COL_NAMES = ["parser_postscores_pilot_bent", "parser_postscores_pilot_straight"]

In [ ]:
# Plot, separated by group and model
from pythonlib.dataset.beh_model_comparison import plots_cross_prior_and_model

plots_cross_prior_and_model(D.Dat, GROUPING, GROUPING_LEVELS, SCORE_COL_NAMES)

#### Scratch

In [ ]:
def score_function(parses, ver="ink", normalization = "inverse", test=False,
                  use_torch=False, origin=None):
    """ 
    - ver, str, determines what score to use
    --- "ink", then total distnace traveled on page
    --- "travel", then total distance traveled, including
    gaps, starting from position of first touch.
    - normalization, how to normalize raw distnace. distance will
    be that more positive is more worse. 
    --- inverse take inverse, so that now less positive is worse.
    --- negative, ...
    """
    from pythonlib.drawmodel.features import strokeDistances, computeDistTraveled

    if test:
        # then just return random number, one for each parse
        return torch.tensor([random.random() for _ in range(len(parses))])    
    
    if ver=="ink":
        # === Total ink used
        distances = [np.sum(strokeDistances(strokes)) for strokes in parses]
    elif ver=="travel":
        # conisder origin to be onset of first storke.
        # Note: has issue in that a single stroke task, flipped, is idnetical cost to the same task unflipped.
        # leads to problems later since unique(score) is used to throw out redundant parses.
        distances_traveled = [computeDistTraveled(strokes, origin=strokes[0][0,[0,1]]) for strokes in parses]
        distances = distances_traveled
    elif ver=="travel_from_orig":
        # pass in origin. 
        assert origin is not None, " must pass in coordinate for origin"
        distances_traveled = [computeDistTraveled(strokes, origin=origin) for strokes in parses]
        distances = distances_traveled

    elif ver=="nstrokes":
        # num strokes
        # == plit histogram of num strokes
        nstrokes = [len(p) for p in parses]        
    else:
        print(ver)
        assert False, "not codede"
        
    if use_torch:
        distances = torch.tensor(distances)
    else:
        distances = np.array(distances)
        
    if normalization=="inverse":
        return 1/distances
    elif normalization=="negative":
        return -distances
    else:
        print(normalization)
        assert False, "not coded"

## GENERATIVE MODEL (LIKE BPL)

In [ ]:
from pythonlib.behmodel.drawmodel.model.model import CharacterModel as DrawModel
from pythonlib.behmodel.drawmodel.library.library import Library
from pythonlib.bpl.strokesToProgram import plotMP

lib = Library()
DM = DrawModel(lib)
ctype = DM.sample_type()
plotMP(ctype)

In [ ]:
## INFERENCE

# 1) For each parsed stroke, assign to max likeli using mixture model.
# OR:
# 1) assign a prob based on mixture model (posterior score)

# 2) Fit model using similar method to Lake

